In [1]:
# Notebook-spezifische Einstellungen
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
import pandas as pd
import plotly.express as px
import plotly.io as pio

# Standard-Theme für Plotly
pio.templates.default = "plotly_dark"

# Pfad zur kombinierten CSV 
DATA_PATH = "../data/cleaned-data/ckw_opendata_smartmeter_dataset_b_all.csv"

# Daten laden (Annahme: es ist ein CSV, kein CSV.GZ mehr)
df = pd.read_csv(DATA_PATH)

# Erste Zeilen anschauen
df.head(20)

,area_code,timestamp,num_meter,value_kwh
0,6221,2021-03-09T22:15:00.000Z,184,10.750000
1,6166,2021-03-05T06:45:00.000Z,884,86.545710
2,6166,2021-03-27T10:45:00.000Z,885,136.944710
3,6214,2021-03-07T15:30:00.000Z,310,26.292000
4,6130,2021-03-02T12:30:00.000Z,3993,360.981178
5,6033,2021-03-24T22:45:00.000Z,106,11.528000
6,6106,2021-03-23T14:30:00.000Z,406,38.629000
7,6166,2021-03-10T14:30:00.000Z,884,79.082710
8,6030,2021-03-27T23:00:00.000Z,884,67.787000
9,6170,2021-03-06T00:45:00.000Z,2147,372.849776


In [2]:
df["timestamp"] = pd.to_datetime(df["timestamp"])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16459651 entries, 0 to 16459650
Data columns (total 4 columns):
 #   Column     Dtype              
---  ------     -----              
 0   area_code  int64              
 1   timestamp  datetime64[ns, UTC]
 2   num_meter  int64              
 3   value_kwh  float64            
dtypes: datetime64[ns, UTC](1), float64(1), int64(2)
memory usage: 502.3 MB


In [3]:
# Check auf fehlende Werte
print(f"Anzahl Zeilen: {len(df)}")
print(df.isna().sum())  # Anzahl fehlender Werte pro Spalte


Anzahl Zeilen: 16459651
area_code    0
timestamp    0
num_meter    0
value_kwh    0
dtype: int64


In [4]:
# Summe des Verbrauchs pro Tag
df_daily = df.groupby(df["timestamp"].dt.date)["value_kwh"].sum().reset_index()
df_daily.columns = ["date", "total_kwh"]

fig = px.line(df_daily, x="date", y="total_kwh", title="Tägliche Gesamt-Verbrauch (kWh)")
fig.show()


In [18]:
# Erstelle eine neue Spalte 'month' (als Periode, z.B. 2025-02)
df["month"] = df["timestamp"].dt.to_period("M")

# Berechne den Gesamtverbrauch pro Monat und pro Area Code
df_monthly = df.groupby(["area_code", "month"])["value_kwh"].sum().reset_index()

# Berechne den durchschnittlichen monatlichen Verbrauch pro Area Code
df_area_monthly = df_monthly.groupby("area_code")["value_kwh"].mean().reset_index()
df_area_monthly.columns = ["area_code", "avg_monthly_kwh"]

# Visualisierung mit Plotly Express
fig = px.bar(
    df_area_monthly,
    x="area_code",
    y="avg_monthly_kwh",
    title="Durchschnittlicher monatlicher Verbrauch pro Wohnort",
    labels={
        "area_code": "Anonymisierter Code für Wohnort",
        "avg_monthly_kwh": "Durchschnittlicher Verbrauch (kWh)"
    }
)

# x-Achse auf einen Bereich beschränken
fig.update_layout(xaxis=dict(range=[6000, 6400]))

fig.show()


/tmp/ipykernel_105960/4014933963.py:2: UserWarning:

Converting to PeriodArray/Index representation will drop timezone information.

